# Solving differential equations in Julia

## Solve ODEs using DifferentialEquations.jl

Documentation: <https://diffeq.sciml.ai/dev/index.html>

In [ ]:
using DifferentialEquations
using Plots
Plots.default(linewidth=2)

### Exponential decay model

The concentration of a decaying nuclear isotope could be described as an exponential decay:

$$
\frac{d}{dt}C(t) = - \lambda C(t)
$$

**State variable**
- $C(t)$: The concentration of a decaying nuclear isotope.

**Parameter**
- $\lambda$: The rate constant of decay. The half-life $t_{\frac{1}{2}} = \frac{ln2}{\lambda}$

In [ ]:
# Model function, in the out-of-place form f(u, p, t)
expdecay(u, p, t) = p * u

p = -1.0            # Parameter of exponential decay
u0 = 1.0            # Initial condition
tspan = (0.0, 2.0)  # Start time and end time

# Define a problem
prob = ODEProblem(expdecay, u0, tspan, p)

# Solve the problem
sol = solve(prob)

# Visualize the solution
plot(sol, legend=:right)

### SIR model

This more complex model simulates the spreading of an contagious disease can be described by the [SIR model](https://www.maa.org/press/periodicals/loci/joma/the-sir-model-for-spread-of-disease-the-differential-equation-model):

$$
\begin{align}
\frac{d}{dt}S(t) &= - \beta S(t)I(t)  \\
\frac{d}{dt}I(t) &= \beta S(t)I(t)  - \gamma I(t)  \\
\frac{d}{dt}R(t) &= \gamma I(t)
\end{align}
$$

**State variables**

- $S(t)$ : the fraction of susceptible people
- $I(t)$ : the fraction of infectious people
- $R(t)$ : the fraction of recovered (or removed) people

**Parameters**

- $\beta$ : the rate of infection when susceptible and infectious people meet
- $\gamma$ : the rate of recovery of infectious people

In [ ]:
# SIR model
function sir!(D, u, p ,t)
	s, i, r = u
	β, γ = p
	v1 = β * s * i
	v2 = γ * i
    D[1] = -v1
    D[2] = v1 - v2
    D[3] = v2
	return nothing
end

# Parameters of the SIR model
p = (β = 1.0, γ = 0.3)
u0 = [0.99, 0.01, 0.00]  # s, i, r
tspan = (0.0, 20.0)

# Define a problem
prob = ODEProblem(sir!, u0, tspan, p)

# Solve the problem
sol = solve(prob)

# Visualize the solution
plot(sol, label=["S" "I" "R"], legend=:right)

### Save simulation results

In [ ]:
using DataFrames
using CSV

In [ ]:
df = DataFrame(sol)

In [ ]:
CSV.write("sir.csv", df)

## Using ModelingToolkit.jl

[ModelingToolkit.jl](https://mtk.sciml.ai/dev/) is a high-level package for symbolic-numeric modeling and simulation ni the Julia DiffEq ecosystem.

In [ ]:
using DifferentialEquations
using ModelingToolkit
using Plots
Plots.default(linewidth=2)

### Exponential decay model

In [ ]:
@parameters λ       # Decaying rate constant
@variables t C(t)   # Time and concentration

D = Differential(t) # Differential operator

# Make an ODE system
@named expdecaySys = ODESystem([D(C) ~ -λ*C ])

In [ ]:
u0 = [C => 1.0]
p = [λ => 1.0]
tspan = (0.0, 2.0)

prob = ODEProblem(expdecaySys, u0, tspan, p)
sol = solve(prob)

plot(sol)

### SIR model

In [ ]:
@parameters β γ
@variables t s(t) i(t) r(t)

D = Differential(t) # Differential operator

# Make an ODE system
@named sirSys = ODESystem(
    [D(s) ~ -β * s * i,
     D(i) ~ β * s * i - γ * i,
     D(r) ~ γ * i])

In [ ]:
# Parameters of the SIR model
p = [β => 1.0, γ => 0.3]
u0 = [s => 0.99, i => 0.01, r => 0.00]
tspan = (0.0, 20.0)

prob = ODEProblem(sirSys, u0, tspan, p)
sol = solve(prob)

plot(sol)

## Using Catalyst.jl

[Catalyst.jl](https://github.com/SciML/Catalyst.jl) is a domain-specific language (DSL) package to solve "law of mass action" problems.

In [ ]:
using Catalyst
using DifferentialEquations
using Plots
Plots.gr(linewidth=2)

### Exponential decay model

In [ ]:
decay_rn = @reaction_network begin
    λ, C --> 0
end

In [ ]:
p = [:λ => 1.]
u0 = [:C => 1.]
tspan = (0., 2.)

prob = ODEProblem(decay_rn, u0, tspan, p)
sol = solve(prob)

plot(sol, title="Exponential Decay")

### SIR model

In [ ]:
sir_rn = @reaction_network begin
    β, S + I --> 2I
    γ, I --> R
end

In [ ]:
# Parameters of the SIR model
p = [:β => 1.0, :γ => 0.3]
u0 = [:S => 0.99, :I => 0.01, :R => 0.00]
tspan = (0., 20.)

prob = ODEProblem(sir_rn, u0, tspan, p)
sol = solve(prob)

plot(sol, legend=:right, title = "SIR Model")

## Runtime information

In [ ]:
versioninfo()

In [ ]:
using Pkg
Pkg.status()